# Swap Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [7]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Valuing a Vanilla IRS using curve assignment to control the discount curve 

This sample value an existing swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the market value in cash (CleanMarketValueInDealCcy, MarketValueInDealCcy, AccruedAmountInDealCcy)
- their equivalent in in percent (AccruedPercent,CleanPricePercent,DirtyPricePercent,CleanPricePercent)
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)

In addition to the swap definition, the sections "curveAssignment" and "marketData" are used
- the "marketDataAssignments" section specifies that the curve "eur_ois_on_curve" ("assignmentTag") is used to discount EUR denominated cash flows.
- the "marketdata" section provide the definition of the curves. the "assignmentTag" is link between the two sections

To learn more about curves, please refer to the samples in /curves-and-surfaces/curves/ 

In [10]:
request_body = {
    
    "fields" : ["InstrumentTag","MarketValueInDealCcy","DirtyPricePercent","FixedRatePercent","SpreadBp","PV01Bp",\
                "PV01AmountInDealCcy","DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Swap",
            "instrumentDefinition": {
                "instrumentTag":"IRS-EURIBOR-5Y",
                "startDate":"2020-01-28",
                "tenor":"5Y",
                "legs":[
                {
                    "direction":"Paid",
                    "interestType":"Fixed",
                    "fixedRatePercent":0.5,
                    "notionalCcy":"EUR",
                    "interestPaymentFrequency":"Annual",
                    "interestCalculationMethod":"Dcb_30_360"
                },
                {
                    "direction":"Received",
                    "interestType":"Float",
                    "interestPaymentFrequency":"SemiAnnual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "indexName":"EURIBOR",
                    "indexTenor":"6M"
                }]
            }
        }],
    
    "pricingParameters": {
        "valuationDate": "2020-10-20T00:00:00Z"
    },
        
    "marketDataAssignments": {
        "rates": {
            "discount": [
            {
                "key": {
                    "currency": "EUR"
                },
                "assignmentTag": "eur_ois_on_curve"
            }]
        }
    },

    "marketData": {
        "interestRateCurves": [
        {
            "assignmentTag": "eur_ois_on_curve",
            "curveDefinition": {
                "currency": "EUR",
                "discountingTenor": "OIS",
                "indexTenor": "OIS"
            }
        }]
    },
     "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'DirtyPricePercent'}, {'type': 'Float', 'name': 'FixedRatePercent'}, {'type': 'Float', 'name': 'SpreadBp'}, {'type': 'Float', 'name': 'PV01Bp'}, {'type': 'Float', 'name': 'PV01AmountInDealCcy'}, {'type': 'String', 'name': 'DiscountCurveName'}, {'type': 'String', 'name': 'ForwardCurveName'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['IRS-EURIBOR-5Y', 25321.3163311753, 2.53213163311753, 0.5, None, 0.058172760929483, 5.8172760929483, 'EUR - OIS EONIA', None, '', ''], ['IRS-EURIBOR-5Y', 25321.3163311753, 2.53213163311753, None, 101.661832607035, -4.00826736398751, -400.826736398751, 'EUR - OIS EONIA', 'EUR - Swap vs 6M Euribor', '', '']]}


In [11]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,MarketValueInDealCcy,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,25321.316331,2.532132,0.5,NaN,0.058173,5.817276,EUR - OIS EONIA,None,,
1,IRS-EURIBOR-5Y,25321.316331,2.532132,NaN,101.661833,-4.008267,-400.826736,EUR - OIS EONIA,EUR - Swap vs 6M Euribor,,
